# Leverage calculations from market data

A simple notebook showing leverage calculations based on market data obtined via the API.

Resources:
- [Testnet](https://console.fairground.wtf/)
- [API docs](https://docs.fairground.vega.xyz/docs/apis/)

## [gRPC](https://docs.fairground.vega.xyz/docs/apis/grpc/)

Uncomment the cell below to install the [Vega-API-client](https://pypi.org/project/Vega-API-client/) for Python if you've not done it yet.

In [2]:
# !pip install --upgrade Vega-API-client

In [3]:
import vegaapiclient as vac
import math
node_url_grpc="n06.testnet.vega.xyz:3007"
data_client = vac.VegaTradingDataClient(node_url_grpc)

def round_sd(number):
    significant_digits = 4
    return round(number, significant_digits - math.floor(math.log10(abs(number))) - 1)

### Specify public key for the party

Specify the your public key in the cell below. It can be found in the [wallet side panel](https://console.fairground.wtf/wallet/) in the Console.

![](./resources/pubkey.png)

In [4]:
pubkey="73ce9e7fbda4e8ae4dec66e9786c17fd7a31a32a616b3287615b421877e3a8a6"
party_positions = data_client.PositionsByParty(vac.data_node.api.v1.trading_data.PositionsByPartyRequest(party_id=pubkey))
if len(party_positions.positions) == 0:
    print("party with the public key specified has no open positions")
    exit()


In [6]:
party_accounts = data_client.PartyAccounts(vac.data_node.api.v1.trading_data.PartyAccountsRequest(party_id=pubkey))

for pos in party_positions.positions:
    market=data_client.MarketByID(vac.data_node.api.v1.trading_data.MarketByIDRequest(market_id=pos.market_id)).market
    market_name=market.tradable_instrument.instrument.name
    market_data=data_client.MarketDataByID(vac.data_node.api.v1.trading_data.MarketDataByIDRequest(market_id=pos.market_id)).market_data
    margin_acc = next(x for x in party_accounts.accounts if x.market_id == pos.market_id )
    gen_acc = next(x for x in party_accounts.accounts if x.type == vac.vega.vega.ACCOUNT_TYPE_GENERAL and x.asset==margin_acc.asset)
    
    asset_dp = data_client.AssetByID(vac.data_node.api.v1.trading_data.AssetByIDRequest(id=gen_acc.asset)).asset.details.decimals
    market_dp = market.decimal_places
    
    mar_acc_bal = float(margin_acc.balance) / asset_dp
    gen_acc_bal = float(gen_acc.balance) / asset_dp

    notional=pos.open_volume*float(market_data.mark_price)/market_dp
    current_leverage=round_sd(notional/(gen_acc_bal+mar_acc_bal))
    capital_utilisation=round_sd((mar_acc_bal/gen_acc_bal)*100)

    margin_levels=data_client.MarginLevels(vac.data_node.api.v1.trading_data.MarginLevelsRequest(party_id=pubkey, market_id=pos.market_id)).margin_levels
    mar_mant = float(margin_levels[0].maintenance_margin) / market_dp
    mar_srch = float(margin_levels[0].maintenance_margin) / market_dp
    mar_init = float(margin_levels[0].initial_margin) / market_dp
    mar_rel = float(margin_levels[0].collateral_release_level) / market_dp

    print("\tmarket: {name}".format(name=market_name)) 
    print("\t\teffective position leverage: ~{lev}x".format(lev=current_leverage))
    print("\t\tleverage per margin level:")
    print("\t\t\t{:<20} ~{}x".format("maintenance:",round_sd(notional/mar_mant)))
    print("\t\t\t{:<20} ~{}x".format("search",round_sd(notional/mar_srch)))
    print("\t\t\t{:<20} ~{}x".format("initial",round_sd(notional/mar_init)))
    print("\t\t\t{:<20} ~{}x".format("release",round_sd(notional/mar_rel)))
    print("\t\tcapital utilisation: ~{lev}%".format(lev=capital_utilisation))  
    print("\t\t\t{:<20} ~{}%".format("maintenance:",round_sd(mar_mant)/notional*100))
    print("\t\t\t{:<20} ~{}%".format("search",round_sd(mar_srch)/notional*100))
    print("\t\t\t{:<20} ~{}%".format("initial",round_sd(mar_init)/notional*100))
    print("\t\t\t{:<20} ~{}%".format("release",round_sd(mar_rel)/notional*100))


[maintenance_margin: "6259204"
search_level: "6885124"
initial_margin: "9388806"
collateral_release_level: "10640646"
party_id: "73ce9e7fbda4e8ae4dec66e9786c17fd7a31a32a616b3287615b421877e3a8a6"
market_id: "c68c3af7119f87a17c3225bbc33814a3d9163a71395a199b700fa5d742d90726"
asset: "8b52d4a3a4b0ffe733cddbc2b67be273816cfeb6ca4c8b339bac03ffba08e4e4"
timestamp: 1654605727973611252
]
	market: Tesla Quarterly (30 Jun 2022)
		effective position leverage: ~-0.5651x
		leverage per margin level:
			maintenance:         ~-42.42x
			search               ~-42.42x
			initial              ~-28.28x
			release              ~-24.95x
		capital utilisation: ~2.057%
			maintenance:         ~-2.3578593848897795%
			search               ~-2.3578593848897795%
			initial              ~-3.536789077334669%
			release              ~-4.007607644604993%
[maintenance_margin: "368893"
search_level: "405782"
initial_margin: "553339"
collateral_release_level: "1180457"
party_id: "73ce9e7fbda4e8ae4dec66e9786c17fd7a31a32a6

## [GraphQL](https://docs.fairground.vega.xyz/docs/apis/graphql/)

All the data required for above calculations can also be obtained via the GraphQL API. Try going to the [GraphQL playground](https://lb.testnet.vega.xyz/playground) for the testnet and entering the query below (remember to modify the public key to yours) 

```bash
query($pubKey: ID = "e23f51fa4236ac4c843b56fed310db713055dae5d5ce71b976465295206b11bf") {        
  party(id:$pubKey){
    positions {
      market {
        name,
      	data {
          markPrice
        }}
      openVolume,
      margins {
        maintenanceLevel
       	searchLevel,
        initialLevel,
        collateralReleaseLevel,
      }
    },
    accounts {
      type,
      balance,
      asset {
        name
      }, 
      market {
        name
      }
    }
  }
}
```